In [1]:
from build123d import *
from ocp_vscode import *
import numpy as np
from IPython.display import display


Make a castle wall

In [25]:
from build123d import *
from ocp_vscode import show_all, show
import numpy as np

height = 10.0 * M
length = 50.0 * M
width = 20.0 * M
bend_radius = 6.0 * M
thickness = 0.5 * M
fillet_radius = 12 * CM
crenel_height = 60 * CM
crenel_width = 1.0 * M
crenellation_ratio = 30

# Create the wall path and extrude it
path = FilletPolyline((0, 0), (length/2, 0), (length/2, height), radius=bend_radius)
wall = path + offset(path, amount=thickness, side=Side.LEFT)
profile = make_face(wall.edges())
profile += mirror(profile, about=Plane.YZ)
fortress_wall = extrude(profile, height)

# Mirror the path for crenel placements on top edge
top_edge = Pos(0, 0, height) * path
top_edge += mirror(top_edge, about=Plane.YZ)

# Loop to place crenels along the top edge
for t in np.linspace(0, 1, crenellation_ratio):
    tangent_vector = Vector(top_edge % t)  # Get tangent vector at position `t`
    tangent_angle = np.degrees(np.arctan2(tangent_vector.Y, tangent_vector.X))  # Calculate angle with atan2
    
    crenel = Box(crenel_width, 3*thickness, crenel_height, align=Align.CENTER)
    fortress_wall -= Pos(top_edge @ t - (0, 0, crenel_height/2)) * Rot(0, 0, tangent_angle) * crenel

# show(profile, fortress_wall, top_edge)
show(fortress_wall)




+

Instance and reshape a few items

In [6]:
from build123d import *
from ocp_vscode import show
import numpy as np
import copy

cutout = Box(1.1, 1.1, 1.1)
l1 = Line((0, 0), (0, 8))
# Create a sphere
base_sphere = Sphere(0.75)

# Do operations on the sphere
weird_ball = base_sphere.cut(cutout)
# Note that you can do affine transformation on the geometry using `Matrix()``
weird_ball =  weird_ball.transform_geometry(Matrix([[0.5, 0, 0, 0], [0, 0.7, 0, 0], [0, 0, 1.0, 0], [0, 0, 0, 1]]))
weird_ball = weird_ball.rotate(Axis.X, 30.0)

# Generate locations as Location objects
locs = [Location(l1 @ t) for t in np.linspace(0.0, 1.0, 7)]
# Create spheres at each location
things = [copy.copy(weird_ball).locate(loc) for loc in locs]


show(things)

------+

In [10]:
from build123d import *
from ocp_vscode import show_all, show

thickness = 3 * MM
width = 25 * MM
length = 50 * MM
height = 25 * MM
hole_diameter = 5 * MM
bend_radius = 5 * MM
fillet_radius = 2 * MM

with BuildPart() as bracket:
    with BuildSketch() as sketch:
        with BuildLine() as profile:
            FilletPolyline(
                (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
            )
            offset(amount=thickness, side=Side.LEFT)
        make_face()
        mirror(about=Plane.YZ)
    extrude(amount=width / 2)
    mirror(about=Plane.XY)
    corners = bracket.edges().filter_by(Axis.X).group_by(Axis.Y)[-1]
    fillet(corners, fillet_radius)
    with Locations(bracket.faces().sort_by(Axis.X)[-1]):
        Hole(hole_diameter / 2)
    with BuildSketch(bracket.faces().sort_by(Axis.Y)[0]):
        SlotOverall(20 * MM, hole_diameter)
    extrude(amount=-thickness, mode=Mode.SUBTRACT)
if bracket.part.is_manifold:
    print("🔒 Part is manifold")
else:
    print("🔓 Part is NOT manifold")
show(bracket)

🔒 Part is manifold
c

In [9]:
from build123d import *
from ocp_vscode import show_all, show

ankh = Bezier((0.0, -14.0),(-2.75, -14.0),(-5.0, -11.75),(-5.0, -9.0))
ankh += Bezier((ankh @ 1),(-5.0, -6.75),(-3.5, -4.5),(-2.5, -3.0))
ankh += Line((ankh @ 1),(-8.0, -3.0))
ankh += Line((ankh @ 1),(-8.0, 0.25))
ankh += Bezier((ankh @ 1),(-5.75, -0.25),(-3.5, -0.5),(-1.0, -0.75))
ankh += Bezier((ankh @ 1),(-1.5, 4.25),(-1.75, 9.0),(-2.0, 14.0))
ankh += Line((ankh @ 1),(0.0, 14.0))
ankh = make_face(ankh+mirror(ankh,Plane.YZ))

ankh_hole = Bezier((0.0, -3.0),(-0.25, -3.25),(-3.0, -6.75),(-3.0, -9.0))
ankh_hole += Bezier(ankh_hole @ 1,(-3.0, -10.75),(-1.75, -12.0),(0.0, -12.0))
ankh_hole = make_face(ankh_hole+mirror(ankh_hole,Plane.YZ))

ankh -= ankh_hole

ankh = extrude(ankh,2)

top_edges = ankh.edges().filter_by(Axis.Z)
ankh = fillet(top_edges,0.3)

top_edges = ankh.edges().group_by(Axis.Z)[-1]
ankh = fillet(top_edges,0.9)
show(ankh)

c

In [11]:
f = Triangle(a=8, b=12, c=13) - Rectangle(1.5, 1.75) 
f = f - Rectangle(1.0, 1.25).translate((0, 2, 0))
f = f - Rectangle(1.5, 2.0).rotate(Axis.Z, 90).translate((0, -2.25, 0))
inside_verts = f.vertices().filter_by(lambda v: v in f.face().inner_wires().vertices())
rounded = fillet(inside_verts, 0.25)
verts = rounded.vertices().filter_by(lambda v: v in f.face().outer_wire().vertices())
rounded = fillet(verts, 1)
plate = extrude(rounded, 1)
# show(inside_verts, verts, rounded)
horizontal_edges = plate.edges().filter_by(Plane.XY)
smoothed = fillet(plate.edges().filter_by(Plane.XY), 0.22)
show(smoothed, inside_verts, horizontal_edges)
export_gltf(smoothed, "smoothed_plate.gltf")

+

True

In [29]:
from build123d import *
from ocp_vscode import *

f = Polyline((-10, 0), (0, 10), (10, 0), (-17, -15), close=True)
h = Polyline((-3.0, 0), (0, 4.5), (3.0, 0), close=True)

# Create faces from the polylines
outer_face = make_face(f)
inner_face = make_face(h)

# Select the vertices from each face for independent filleting
outer_verts = outer_face.vertices()
inner_verts = inner_face.vertices()

outer_face = fillet(outer_verts, 1.25)
inner_face = fillet(inner_verts, 0.25)

# Extrude each face to create thin 3D solids
outer_solid = extrude(outer_face, 1)
inner_solid = extrude(inner_face, 1)

carved_solid = outer_solid - inner_solid
carved_solid = fillet(carved_solid.edges(), 0.25)

show(carved_solid)


+

In [4]:
from build123d import *
from ocp_vscode import *

f = Polyline((-10, 0), (0, 10), (10, 0), (-17, -15), close=True)
h = Polyline((-3.0, 0), (0, 4.5), (3.0, 0), close=True)

# Create faces from the polylines
outer_face = make_face(f)
inner_face = make_face(h)

# Select the vertices from each face for independent filleting
outer_verts = outer_face.vertices()
inner_verts = inner_face.vertices()

# Round off the inside corners first
outer_face = fillet(outer_verts, 1.25)
inner_face = fillet(inner_verts, 0.25)
face = outer_face - inner_face

# Extrude each face to create thin 3D solids
carved_solid = extrude(face, 2.0)
carved_solid = fillet(carved_solid.edges(), 0.25)

show(carved_solid, inner_verts, outer_verts)
if carved_solid.is_manifold:
    print("🔒 Part is manifold")
else:
    print("🔓 Part is non-manifold")

c🔒 Part is manifold
